In [3]:

%matplotlib widget

### 累計レポート
# 標準ライブラリ
import os
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Callable

# サードパーティライブラリ
import pandas as pd
from pandas import Timedelta
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from IPython.display import Markdown, display

# プロジェクト内モジュール
from gamedata import hard_sales as hs
from gamedata import plot_hard as ph
from gamedata import hard_info as hi
from gamedata import hard_event as he
from gamedata import chart_hard as ch

plt.show()

# ランキングダッシュボード

## 週販ランキング

In [4]:
df_all = hs.load_hard_sales()
df_all

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,adjust_units,year,month,...,week,delta_day,delta_week,delta_month,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
0,1998-10-18_GB,1998-10-12,1998-10-18,1998-10-18,7,GB,10867,0,1998,10,...,3,3467,495,114,9,1552,10867,1989-04-21,Nintendo,GAME BOY
1,1998-10-18_N64,1998-10-12,1998-10-18,1998-10-18,7,N64,2965,0,1998,10,...,3,847,121,28,2,423,2965,1996-06-23,Nintendo,NINTENDO64
2,1998-10-18_PS,1998-10-12,1998-10-18,1998-10-18,7,PS,20918,0,1998,10,...,3,1415,202,46,4,2988,20918,1994-12-03,SONY,PlayStation
3,1998-10-18_SATURN,1998-10-12,1998-10-18,1998-10-18,7,SATURN,858,0,1998,10,...,3,1426,203,47,4,122,858,1994-11-22,SEGA,SEGA SATURN
4,1998-10-25_GB,1998-10-19,1998-10-25,1998-10-25,7,GB,163017,0,1998,10,...,4,3474,496,114,9,23288,173884,1989-04-21,Nintendo,GAME BOY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8529,2025-11-30_NS2,2025-11-24,2025-11-30,2025-11-30,7,NS2,97677,0,2025,11,...,5,178,25,5,0,13953,2968435,2025-06-05,Nintendo,Nintendo Switch2
8530,2025-11-30_NSW,2025-11-24,2025-11-30,2025-11-30,7,NSW,35327,0,2025,11,...,5,3194,456,104,8,5046,36262542,2017-03-03,Nintendo,Nintendo Switch
8531,2025-11-30_PS4,2025-11-24,2025-11-30,2025-11-30,7,PS4,13,0,2025,11,...,5,4299,614,141,11,1,9490987,2014-02-22,SONY,PlayStation4
8532,2025-11-30_PS5,2025-11-24,2025-11-30,2025-11-30,7,PS5,35786,0,2025,11,...,5,1844,263,60,5,5112,7215519,2020-11-12,SONY,PlayStation5


In [11]:
hs.weekly_sales(df_all)

,report_date,hw,weekly_units,sum_units
0,1998-10-18,GB,10867,10867
1,1998-10-18,N64,2965,2965
2,1998-10-18,PS,20918,20918
3,1998-10-18,SATURN,858,858
4,1998-10-25,GB,163017,173884
...,...,...,...,...
8529,2025-11-30,NS2,97677,2968435
8530,2025-11-30,NSW,35327,36262542
8531,2025-11-30,PS4,13,9490987
8532,2025-11-30,PS5,35786,7215519


### 総合ランキング

In [25]:
# df_all[units]が多いい順に並べる
df_sorted = df_all.sort_values(by='units', ascending=False)
# df_sortedのTOP10を表示, 表示カラムは report_date, hw, untis, sum_units
df_sorted_10 = df_sorted.head(10)[['report_date', 'hw', 'units', 'sum_units']]
# indexをリセット 1から始まるようにする
df_sorted_10 = df_sorted_10.reset_index(drop=True)
df_sorted_10.index += 1
df_sorted_10

,report_date,hw,units,sum_units
1,2025-06-08,NS2,715432,715432
2,2000-03-05,PS2,630552,630552
3,2001-03-25,GBA,611504,611504
4,2006-01-01,DS,511472,5506996
5,2005-12-25,DS,511471,4995524
6,2011-12-25,3DS,510629,4132038
7,2004-12-05,DS,441485,441485
8,2012-12-23,3DS,433788,9492011
9,2020-03-22,NSW,392576,12801240
10,2011-12-18,3DS,387838,3621409


In [ ]:
def chart_weekly_ranking(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], maker:List[str] = []) -> pd.DataFrame:
    """週間ランキングチャートを出力

    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    df_weekly = hs.load_hard_sales()
    if begin:
        df_weekly = df_weekly[df_weekly['report_date'] >= begin]
    if end:
        df_weekly = df_weekly[df_weekly['report_date'] <= end]
        
    if rank_n < 0:
        top_n = abs(rank_n)
        ascending = True
    else:
        top_n = rank_n
        ascending = False
    df_weekly_sorted = df_weekly.sort_values(by='units', ascending=ascending)
        
    if hw:
        df_weekly_sorted = df_weekly_sorted[df_weekly_sorted['hw'].isin(hw)]
    if maker:
        df_weekly_sorted = df_weekly_sorted[df_weekly_sorted['maker_name'].isin(maker)]
    df_weekly_sorted_10 = df_weekly_sorted.head(top_n)[['report_date', 'hw', 'units', 'sum_units']]
    df_weekly_sorted_10 = df_weekly_sorted_10.reset_index(drop=True)
    df_weekly_sorted_10.index += 1
    df_weekly_sorted_10 = ch.rename_columns(df_weekly_sorted_10)
    return df_weekly_sorted_10


In [18]:
chart_weekly_ranking(rank_n=10)

,集計日,ハード,販売台数,累計台数
1,2025-06-08,NS2,715432,715432
2,2000-03-05,PS2,630552,630552
3,2001-03-25,GBA,611504,611504
4,2006-01-01,DS,511472,5506996
5,2005-12-25,DS,511471,4995524
6,2011-12-25,3DS,510629,4132038
7,2004-12-05,DS,441485,441485
8,2012-12-23,3DS,433788,9492011
9,2020-03-22,NSW,392576,12801240
10,2011-12-18,3DS,387838,3621409


In [19]:
chart_weekly_ranking(rank_n=-10, hw=['NSW'])

,集計日,ハード,販売台数,累計台数
1,2025-07-20,NSW,11766,35836338
2,2025-06-22,NSW,12769,35778126
3,2025-06-15,NSW,13319,35765357
4,2025-07-13,NSW,13411,35824572
5,2025-07-27,NSW,14643,35850981
6,2025-06-29,NSW,15323,35793449
7,2025-06-08,NSW,16611,35752038
8,2025-09-21,NSW,16612,36005334
9,2025-07-06,NSW,17712,35811161
10,2025-08-24,NSW,17945,35924909


In [45]:
chart_weekly_ranking(rank_n=10, hw=['PS5'])

,report_date,hw,units,sum_units
1,2020-11-15,PS5,118085,118085
2,2025-03-02,PS5,108978,6666457
3,2023-11-12,PS5,107953,4630339
4,2023-02-12,PS5,93574,2798318
5,2023-02-05,PS5,93026,2704744
6,2023-02-26,PS5,91729,2978700
7,2023-02-19,PS5,88653,2886971
8,2024-11-10,PS5,88160,6207882
9,2023-06-25,PS5,85385,3803183
10,2023-03-05,PS5,78412,3057112


In [35]:
chart_weekly_ranking(top_n=10, maker=['Nintendo'])

,report_date,hw,units,sum_units
1,2025-06-08,NS2,715432,715432
2,2001-03-25,GBA,611504,611504
3,2006-01-01,DS,511472,5506996
4,2005-12-25,DS,511471,4995524
5,2011-12-25,3DS,510629,4132038
6,2004-12-05,DS,441485,441485
7,2012-12-23,3DS,433788,9492011
8,2020-03-22,NSW,392576,12801240
9,2011-12-18,3DS,387838,3621409
10,2005-12-18,DS,381776,4484053


## 月版ランキング

### 総合ランキング

In [6]:
monthly_all = hs.monthly_sales(df_all, maker_mode=True)
monthly_all

,year,month,maker_name,monthly_units,sum_units
0,1998,10,Nintendo,180662,180662
1,1998,10,SEGA,1919,1919
2,1998,10,SONY,42069,42069
3,1998,11,Nintendo,367056,547718
4,1998,11,SEGA,105013,106932
...,...,...,...,...,...
1066,2025,10,Nintendo,454479,138727651
1067,2025,10,SONY,64806,80671684
1068,2025,11,Microsoft,1891,2877571
1069,2025,11,Nintendo,638432,139366083


In [15]:
def chart_periodic_ranking(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], 
                         maker:List[str] = [],
                         data_source_fn:Callable = hs.monthly_sales,
                         sort_column:str = 'monthly_units',
                         headers:List[str] = ['year', 'month']
                         ) -> pd.DataFrame:
    """ある期間のランキングチャートを出力

    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    df_all = hs.load_hard_sales()
    if (not hw) and maker:
        # This is maker mode
        maker_mode = True
        key_column = 'maker_name'
    else:
        maker_mode = False
        key_column = 'hw'
        
    df_sum = data_source_fn(df_all, begin=begin, end=end, maker_mode=maker_mode)
        
    if rank_n < 0:
        top_n = abs(rank_n)
        ascending = True
    else:
        top_n = rank_n
        ascending = False
    df_sorted = df_sum.sort_values(by=sort_column, ascending=ascending) 
        
    if hw:
        df_sorted = df_sorted[df_sorted[key_column].isin(hw)]
    if maker:
        df_sorted = df_sorted[df_sorted[key_column].isin(maker)]
        
    df_sorted_top = df_sorted.head(top_n)[[*headers, key_column, sort_column]]
    df_sorted_top = df_sorted_top.reset_index(drop=True)
    df_sorted_top.index += 1
    df_sorted_top = ch.rename_columns(df_sorted_top)
    return df_sorted_top


In [16]:
def chart_weekly_ranking2(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], 
                         maker:List[str] = []) -> pd.DataFrame:
    """週間ランキングチャートを出力
    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    return chart_periodic_ranking(rank_n=rank_n, begin=begin, end=end, 
                                  hw=hw, maker=maker, 
                                  data_source_fn=hs.weekly_sales, 
                                  sort_column='weekly_units', 
                                  headers=['report_date'])

In [11]:
def chart_monthly_ranking2(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], 
                         maker:List[str] = []) -> pd.DataFrame:
    """月間ランキングチャートを出力
    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    return chart_periodic_ranking(rank_n=rank_n, begin=begin, end=end, 
                                  hw=hw, maker=maker, 
                                  data_source_fn=hs.monthly_sales, 
                                  sort_column='monthly_units', 
                                  headers=['year', 'month'])

In [13]:
def chart_yearly_ranking2(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], 
                         maker:List[str] = []) -> pd.DataFrame:
    """年間ランキングチャートを出力
    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    return chart_periodic_ranking(rank_n=rank_n, begin=begin, end=end, 
                                  hw=hw, maker=maker, 
                                  data_source_fn=hs.yearly_sales, 
                                  sort_column='yearly_units', 
                                  headers=['year'])

In [ ]:
chart_weekly_ranking2(rank_n=10)

,集計日,メーカー,weekly_units
1,2025-06-08,Nintendo,732043
2,2000-03-05,SONY,651555
3,2001-03-25,Nintendo,628372
4,2011-12-25,Nintendo,610098
5,2007-12-23,Nintendo,584171
6,2006-01-01,Nintendo,576577
7,2005-12-25,Nintendo,576575
8,2012-12-23,Nintendo,572929
9,2006-12-03,Nintendo,557367
10,2006-12-31,Nintendo,545800


In [15]:
chart_yearly_ranking2(rank_n=10)

,年,ハード,年間販売台数
1,2006,DS,8966021
2,2007,DS,7202068
3,2020,NSW,5956943
4,2012,3DS,5646213
5,2021,NSW,5579127
6,2013,3DS,4912059
7,2022,NSW,4804546
8,2019,NSW,4493903
9,2001,GBA,4245686
10,2011,3DS,4132038


In [12]:
chart_monthly_ranking2(rank_n=10)

,年,月,ハード,月間販売台数
1,2011,12,3DS,1492931
2,2006,12,DS,1477904
3,2012,12,3DS,1427194
4,2005,12,DS,1344442
5,2025,6,NS2,1169684
6,2018,12,NSW,1158743
7,2019,12,NSW,1137642
8,2007,12,DS,1111826
9,2004,12,DS,1056428
10,2006,4,DS,1050970


In [ ]:
def chart_monthly_ranking(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], 
                         maker:List[str] = []) -> pd.DataFrame:
    """月間ランキングチャートを出力

    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    df_all = hs.load_hard_sales()
    if (not hw) and maker:
        # This is maker mode
        maker_mode = True
        key_column = 'maker_name'
    else:
        maker_mode = False
        key_column = 'hw'
        
    df_sum = hs.monthly_sales(df_all, begin=begin, end=end, maker_mode=maker_mode)
        
    if rank_n < 0:
        top_n = abs(rank_n)
        ascending = True
    else:
        top_n = rank_n
        ascending = False
    df_sorted = df_sum.sort_values(by='monthly_units', ascending=ascending) 
        
    if hw:
        df_sorted = df_sorted[df_sorted[key_column].isin(hw)]
    if maker:
        df_sorted = df_sorted[df_sorted[key_column].isin(maker)]
        
    df_sorted_top = df_sorted.head(top_n)[['year', 'month', key_column, 'monthly_units']]
    df_sorted_top = df_sorted_top.reset_index(drop=True)
    df_sorted_top.index += 1
    df_sorted_top = ch.rename_columns(df_sorted_top)
    return df_sorted_top


In [ ]:
def chart_yearly_ranking(rank_n:int = 10, 
                         begin:Optional[datetime] = None, 
                         end:Optional[datetime] = None,
                         hw:List[str] = [], 
                         maker:List[str] = []) -> pd.DataFrame:
    """年間ランキングチャートを出力

    Args:
        rank_n (int, optional): 上位n件. Defaults to 10.
            マイナスだった場合は下位n件.
        begin (Optional[datetime], optional): 開始日. Defaults to None.
        end (Optional[datetime], optional): 終了日. Defaults to None.
        hw (List[str], optional): ハード名リスト. Defaults to [].
        maker (List[str], optional): メーカー名リスト. Defaults to [].
    """
    df_all = hs.load_hard_sales()
    if (not hw) and maker:
        # This is maker mode
        maker_mode = True
        key_column = 'maker_name'
    else:
        maker_mode = False
        key_column = 'hw'
        
    df_sum = hs.yearly_sales(df_all, begin=begin, end=end, maker_mode=maker_mode)
        
    if rank_n < 0:
        top_n = abs(rank_n)
        ascending = True
    else:
        top_n = rank_n
        ascending = False
    df_sorted = df_sum.sort_values(by='yearly_units', ascending=ascending) 
        
    if hw:
        df_sorted = df_sorted[df_sorted[key_column].isin(hw)]
    if maker:
        df_sorted = df_sorted[df_sorted[key_column].isin(maker)]
        
    df_sorted_top = df_sorted.head(top_n)[['year', key_column, 'yearly_units']]
    df_sorted_top = df_sorted_top.reset_index(drop=True)
    df_sorted_top.index += 1
    df_sorted_top = ch.rename_columns(df_sorted_top)
    return df_sorted_top

In [24]:

monthly_sorted = monthly_all.sort_values(by='monthly_units', ascending=False)
monthly_sorted_10 = monthly_sorted.head(10)[['year', 'month', 'hw', 'monthly_units', 'sum_units']]
monthly_sorted_10 = monthly_sorted_10.reset_index(drop=True)
monthly_sorted_10.index += 1
monthly_sorted_10

,year,month,hw,monthly_units,sum_units
1,2011,12,3DS,1492931,4132038
2,2006,12,DS,1477904,13961545
3,2012,12,3DS,1427194,9778251
4,2005,12,DS,1344442,4995524
5,2025,6,NS2,1169684,1169684
6,2018,12,NSW,1158743,6889546
7,2019,12,NSW,1137642,11383449
8,2007,12,DS,1111826,21163613
9,2004,12,DS,1056428,1056428
10,2006,4,DS,1050970,7569546


In [3]:
chart_monthly_ranking(rank_n=-10, maker=['SONY'], begin=datetime(2015, 1, 1))

,年,月,メーカー,月間販売台数
1,2020,9,SONY,8732
2,2020,7,SONY,13940
3,2020,8,SONY,15357
4,2020,10,SONY,17726
5,2020,2,SONY,27336
6,2025,7,SONY,29050
7,2021,11,SONY,30704
8,2025,5,SONY,32109
9,2019,10,SONY,35129
10,2021,12,SONY,37427


In [4]:
chart_monthly_ranking(rank_n=10, hw=['PS5'])

,年,月,ハード,月間販売台数
1,2023,2,PS5,366982
2,2023,7,PS5,243383
3,2023,1,PS5,234329
4,2023,11,PS5,231317
5,2023,3,PS5,222656
6,2023,4,PS5,219646
7,2025,3,PS5,218300
8,2023,6,PS5,215521
9,2023,12,PS5,212371
10,2020,11,PS5,201435


In [10]:
chart_monthly_ranking(rank_n=10, hw=['NS2'])

,year,month,hw,monthly_units
1,2025,6,NS2,1169684
2,2025,11,NS2,500414
3,2025,7,NS2,444706
4,2025,10,NS2,358399
5,2025,8,NS2,319690
6,2025,9,NS2,175542


## 年間ランキング


### 総合ランキング

In [6]:
chart_yearly_ranking(rank_n=10)

,年,ハード,年間販売台数
1,2006,DS,8966021
2,2007,DS,7202068
3,2020,NSW,5956943
4,2012,3DS,5646213
5,2021,NSW,5579127
6,2013,3DS,4912059
7,2022,NSW,4804546
8,2019,NSW,4493903
9,2001,GBA,4245686
10,2011,3DS,4132038


In [2]:
yearly_all = hs.yearly_sales(df_all)
yearly_sorted = yearly_all.sort_values(by='yearly_units', ascending=False)
yearly_sorted_10 = yearly_sorted.head(10)[['year', 'hw', 'yearly_units', 'sum_units']]
yearly_sorted_10 = yearly_sorted_10.reset_index(drop=True)
yearly_sorted_10.index += 1
yearly_sorted_10

NameError: name 'df_all' is not defined